In [3]:
import os;
def addFiles(fileList, directory):
   for root, dirs, files in os.walk(directory):
        for file in files:
            fileList.append(root+'/'+file)
        for subdir in dirs:
            addFiles(fileList, subdir);

class Requirement:
    def __init__(self, matchIdx, matchValue, captureIndex, foundValue):
        self.matchIdx = matchIdx
        self.matchValue = matchValue
        self.captureIndex = captureIndex
        self.foundValue = foundValue


fileList = []
addFiles(fileList, '/home/matt/Projects/Web3HackerNetwork/data/github');
print('Found '+str(len(fileList))+' files');
for file in fileList:
    firstline = None
    fileExt = None
    linecount = 0
    commitLineIdx = 0
    commitLine = None
    commitID = None
    """
    define an array of matching patterns
    Params:
    1 - Index to expect parameter 2
    2 - Space delimited match string
    3 - Data array index to capture
 - Populated value - starts as None (null)
    """

    reqArray = []
    reqArray.append( Requirement(0, 'commit', 1, None) )
    reqArray.append( Requirement(0, 'Author:', 1, None) )
    
    
    with open(file, 'r') as file:
        for line in file:
            linecount += 1
            
            for req in reqArray:
                if (req.foundValue is None):
                    """ search for it """
                    sp = line.split(' ')
                    if (sp is not None and 
                        len(sp) > req.matchIdx and 
                        len(sp) > req.captureIndex and
                        sp[req.matchIdx] == req.matchValue):
                        req.foundValue = sp[req.captureIndex]
        file.close()
    
"""
    print(str(file)+' : '+str(linecount)+' lines')
    for req in reqArray:
        print('\t'+str(req.matchValue)+'->'+str(req.foundValue));
"""        
    

Found 82 files


"\nWhat the heck?\n    print(str(file)+' : '+str(linecount)+' lines')\n    for req in reqArray:\n        print('\t'+str(req.matchValue)+'->'+str(req.foundValue));\n"